In [1]:
import local_models.local_models
import numpy as np
import scipy.stats
import matplotlib.pyplot as plt
from importlib import reload
from ml_battery.utils import cmap
import matplotlib as mpl
import logging
import ml_battery.log
import time
import os
import functools
import collections
import itertools
import pymongo
import datetime
import pytz
import re
import local_models.loggin
import local_models.tf_w2v_models
import pickle
from nltk.tokenize import TweetTokenizer


logger = logging.getLogger(__name__)

np.random.seed(1)
#reload(local_models.loggin)
#reload(local_models.TLS_models)
np.warnings.filterwarnings('ignore')


/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:529: FutureWarning: Passing (type, 1) or 

In [2]:
mpl.rcParams['figure.figsize'] = [8.0, 8.0]

In [3]:
RUN = 1
BANDWIDTH = 0.35
SEED = 1
np.random.seed(SEED)
KERNEL=local_models.local_models.GaussianKernel
root_dir = "/home/scott/local_w2v/"
project_dir = os.path.join(root_dir, "r{:03d}_k{}".format(RUN, KERNEL(bandwidth=BANDWIDTH)))
data_dir = os.path.join(root_dir, "data")
os.makedirs(project_dir, exist_ok=1)

In [4]:
def remove_by_regex(text, regexp):
    return regexp.sub("", text)

url_re = re.compile(r"http.?://[^\s]+[\s]?")
def remove_urls(text):
    return remove_by_regex(text, url_re)

specialchar_re = re.compile("[" + ''.join(map(re.escape, 
    [",", ":", "\"", "=", "&", ";", "%", "$", "@", "%", "^", "*", "(", ")", "{", "}",
     "[", "]", "|", "/", "\\", ">", "<", "-", "!", "?", ".", "'", "--", "---"])) + "]")
def remove_special_chars(text):
    return remove_by_regex(text, specialchar_re)

username_re = re.compile(r"@[^\s]+[\s]?")
def remove_usernames(text):
    return remove_by_regex(text, username_re)

number_re = re.compile(r"\s?[0-9]+\.?[0-9]*")
def remove_numbers(text):
    return remove_by_regex(text, number_re)
    
def lower(text):
    return text.lower()

cleanup = (remove_urls,
           remove_usernames,
           remove_special_chars,
           remove_numbers,
           lower)

In [5]:
def get_cikm_tweet_column(line):
    try:
        return line.split("\t")[-2]
    except Exception as e:
        return None
def count(iterable, item):
    tot = 0
    for i in iterable:
        if i == item:
            tot += 1
    return tot

In [6]:
pre_training_data = os.path.join(data_dir, "training_set_tweets.txt")
tknzr = TweetTokenizer(strip_handles=True, reduce_len=True) 

In [7]:
class fuckit(object):
    def __init__(self, message=None):
        self.message = message
    def __enter__(self): return self
    def __exit__(self, *args):
        if self.message is not None:
            print(self.message)
        return True

In [8]:
with fuckit("nothing to del"):
    TFGlobalSesh.close()
with fuckit("nothing to del"):
    del TFGlobalSesh 
with fuckit("nothing to del"):
    with TFGlobalGraph.as_default():
        tf.reset_default_graph()
with fuckit("nothing to del"):
    del TFGlobalGraph 
with fuckit("nothing to del"):
    del model

nothing to del
nothing to del
nothing to del
nothing to del
nothing to del


In [9]:
VOCAB_SIZE = 6000
BASE_MODEL_EPOCHS = 10000
BASE_MODEL_RUN = 2

In [10]:
try:
    with open(os.path.join(root_dir, "base_model_v{:08d}_e{:08d}_r{:03d}.model".format(VOCAB_SIZE, BASE_MODEL_EPOCHS, BASE_MODEL_RUN)), 'rb') as f:
        pmodel = pickle.load(f)
except FileNotFoundError as e:
    pmodel = Word2Vec(vocabulary_size=VOCAB_SIZE, epochs=BASE_MODEL_EPOCHS, log_epochs=0.01)
    pmodel.fit(toknized_tweets, sample_weight=np.ones(len(toknized_tweets)))
    import pickle
    with open(os.path.join(root_dir, "base_model_v{:08d}_e{:08d}_r{:03d}.model".format(VOCAB_SIZE, BASE_MODEL_EPOCHS, BASE_MODEL_RUN)), 'wb') as f:
        pickle.dump(model, f)

In [11]:
pmodel.sort("sweet")

['sweet',
 'yummy',
 'lovely',
 'delicious',
 'nice',
 'beautiful',
 'cool',
 'gorgeous',
 'tasty',
 'cute',
 'funny',
 'rare',
 'awesome',
 'wet',
 'fabulous',
 'adorable',
 'sick',
 'goodnight',
 'similar',
 'wonderful',
 'familiar',
 'neat',
 'sad',
 'weird',
 'perfect',
 'good',
 'happy',
 'sexy',
 'welcome',
 'rad',
 'warm',
 'bff',
 'strange',
 'goood',
 'ricky',
 'bright',
 'dope',
 'gd',
 'fly',
 'romantic',
 'lonely',
 'boring',
 'aww',
 'lucky',
 'badass',
 'jealous',
 'nasty',
 'fresh',
 'windy',
 'sleepy',
 'glorious',
 'amazing',
 'impressive',
 'xoxo',
 'creepy',
 'chill',
 'salad',
 'disgusting',
 'honey',
 'cheap',
 'scary',
 'awww',
 'loves',
 'proud',
 'wicked',
 'lame',
 'clever',
 'mmm',
 'hot',
 'dry',
 'spicy',
 'bomb',
 'hilarious',
 'priceless',
 'tgif',
 'friendly',
 'easy',
 'pleased',
 'blessed',
 'painful',
 'loving',
 'safe',
 'dangerous',
 'fantastic',
 'helpful',
 'alright',
 'silly',
 'annoying',
 'stoked',
 'kisses',
 'dirty',
 'fab',
 'chilly',
 'incre

In [12]:
pmodel

Word2Vec(architecture='cbow', batch_size=128,
         count=Counter({'UNK': 6334388, 'the': 1415963, 'to': 1117395,
                        'a': 884380, 'i': 821540, 'and': 624171, 'for': 591703,
                        'in': 585901, 'of': 577948, 'you': 572544, 'is': 523046,
                        'on': 476378, 'it': 387693, 'my': 361536, 'rt': 348880,
                        'at': 310664, 'that': 275618, 'with': 261940,
                        'this': 253464, 'your': 229211, 'be': 217495,
                        'me': 213922, 'just': 206190...
         reverse_dictionary={0: 'UNK', 1: 'ama', 2: 'lesson', 3: 'doll',
                             4: 'soy', 5: 'charlotte', 6: 'issue', 7: 'flight',
                             8: 'lawn', 9: 'demi', 10: 'street', 11: 'drops',
                             12: 'finally', 13: '#ff', 14: 'theatre',
                             15: 'enjoyed', 16: 'winner', 17: 'brands',
                             18: 'community', 19: 'though', 20: 'cyber',


In [17]:
pmodel.epochs = 10
pmodel.batch_size=1

In [18]:
rando_data = local_models.tf_w2v_models.build_dataset_predictionary(
    ["the quick brown fox jumps over the lazy dog".split(" ")],
    pmodel.dictionary, pmodel.reverse_dictionary)

In [19]:
rando_data

[[4684, 3350, 4985, 91, 0, 746, 4684, 1592, 4099]]

In [22]:
pmodel.fit(rando_data, sample_weight=np.array([1]))

Word2Vec(architecture='cbow', batch_size=1,
         count=Counter({'UNK': 6334388, 'the': 1415963, 'to': 1117395,
                        'a': 884380, 'i': 821540, 'and': 624171, 'for': 591703,
                        'in': 585901, 'of': 577948, 'you': 572544, 'is': 523046,
                        'on': 476378, 'it': 387693, 'my': 361536, 'rt': 348880,
                        'at': 310664, 'that': 275618, 'with': 261940,
                        'this': 253464, 'your': 229211, 'be': 217495,
                        'me': 213922, 'just': 206190, '...
         reverse_dictionary={0: 'UNK', 1: 'ama', 2: 'lesson', 3: 'doll',
                             4: 'soy', 5: 'charlotte', 6: 'issue', 7: 'flight',
                             8: 'lawn', 9: 'demi', 10: 'street', 11: 'drops',
                             12: 'finally', 13: '#ff', 14: 'theatre',
                             15: 'enjoyed', 16: 'winner', 17: 'brands',
                             18: 'community', 19: 'though', 20: 'cyber',